In [1]:
%matplotlib inline
import pandas
import os
import collections
import numpy

from train_test import find_files, parse_dates, load_series, load_data, separate_output

path = "../data/v2/train_set/"
power_data = load_series(find_files(path, "power"), resample_interval="1H")

Using Theano backend.


In [2]:
pandas.set_option("display.max_rows", 500)
saaf_data = load_series(find_files(path, "saaf"))
evtf_data = load_series(find_files(path, "evtf"))
dmop_data = load_series(find_files(path, "dmop"))
ftl_data = load_series(find_files(path, "ftl"))

In [3]:
dmop_data.subsystem.value_counts()

AAAAF56A1           20035
AACFE91A            19493
APSF28A1            18312
AACFE03A            18227
APSF38A1            15611
AAAAF20E1           13440
AVVV02A0            10073
AVVV03B0             9774
AAAAF20C1            9205
AMMMF10A0            8938
AACFE05A             8394
AAAAF23G1            6450
AAAAF57A1            5532
AAAAF60A1            5530
AMMMF18A0            5029
AMMMF19A0            5004
APSF32A1             4893
APSF50A2             4854
AACFM07A             4634
AACFM06A             4600
AAAAF93A1            4541
AAAAF59A1            4480
APSF12H1             4076
ASSSF06A0            3981
ASSSF01A0            3980
AMMMF23A0            3974
AMMMF24A0            3962
AACFM01A             3792
AACFM02A             3782
ATTTF030A            3693
ATTTF030B            3628
ATTTF310B            3601
ATTTF310A            3599
AMMMF05A0            3593
AMMMF01A0            3559
AMMMF06A0            3534
AAAAF84A1            3164
AAAAF85A1            3163
AMMMF04A0   

In [10]:
from train_test import get_dmop_subsystem

get_dmop_subsystem(dmop_data).value_counts().index[:100]

Index([u'AAAF56A1', u'ACFE91A', u'PSF28A1', u'ACFE03A', u'PSF38A1',
       u'AAAF20E1', u'VVV02A0', u'VVV03B0', u'AAAF20C1', u'MMMF10A0', u'MAPO',
       u'ACFE05A', u'MPER', u'AAAF23G1', u'AAAF57A1', u'AAAF60A1', u'MMMF18A0',
       u'MMMF19A0', u'PSF32A1', u'PSF50A2', u'ACFM07A', u'MOCE', u'ACFM06A',
       u'MOCS', u'AAAF93A1', u'AAAF59A1', u'PENS', u'PENE', u'PSF12H1',
       u'SSSF06A0', u'SSSF01A0', u'MMMF23A0', u'MMMF24A0', u'ACFM01A',
       u'ACFM02A', u'TTTF030A', u'TTTF030B', u'TTTF310B', u'TTTF310A',
       u'MMMF05A0', u'MMMF01A0', u'MMMF06A0', u'AAAF84A1', u'AAAF85A1',
       u'MMMF04A0', u'MMMF12A0', u'MMMF11A0', u'MMMF40C0', u'MMMF20A0',
       u'PSF60A0', u'PSF60B0', u'AAAF40B0', u'AAAF40C0', u'PSF30C2',
       u'SSSF53A0', u'PSF30A1', u'PSF03A3', u'PSF33A1', u'PSF13A3', u'PSF82A1',
       u'PSF30B2', u'PSF89A1', u'PSF31A1', u'TTTF301F', u'PSF14A2', u'PSF02A1',
       u'PSF40A1', u'PSF31B1', u'AAAF20D1', u'PSF37A1', u'AAAF18D1',
       u'AAAF19D1', u'MMMF40A0', u'PSF23

In [2]:
import pandas
old_data = pandas.read_csv("../data/v2/train.1H.04_17.64_features.csv", index_col=0, parse_dates=True)
new_data = pandas.read_csv("../data/v2/training.1H.04_23.64_features.csv", index_col=0, parse_dates=True)

In [3]:
old_features = set(old_data.columns)
new_features = set(new_data.columns)

shared_features = old_features & new_features
print "Shared features:", sorted(shared_features)
print "Old only:", sorted(old_features - shared_features)
print "New only:", sorted(new_features - shared_features)

Shared features: ['DMOP_event_counts_log', 'DMOP_event_counts_rolling_2h_gradient', 'DMOP_event_counts_rolling_5h', 'EVTF_IN_MAR_UMBRA', 'EVTF_IN_MAR_UMBRA_rolling_1h', 'EVTF_IN_MAR_UMBRA_rolling_1h_rolling_50', 'EVTF_IN_MRB_/_RANGE_06000KM', 'EVTF_IN_MRB_/_RANGE_06000KM_gradient', 'EVTF_IN_MRB_/_RANGE_06000KM_rolling_1h', 'EVTF_IN_MRB_/_RANGE_06000KM_rolling_1h_gradient', 'EVTF_IN_MRB_/_RANGE_06000KM_rolling_1h_rolling_1600', 'EVTF_IN_MSL_/_RANGE_06000KM', 'EVTF_IN_MSL_/_RANGE_06000KM_rolling_1h', 'EVTF_altitude', 'EVTF_event_counts', 'EVTF_event_counts_rolling_2h', 'EVTF_event_counts_rolling_5h', 'EVTF_event_counts_rolling_5h_rolling_50', 'FTL_ACROSS_TRACK', 'FTL_ACROSS_TRACK_rolling_1h', 'FTL_ACROSS_TRACK_rolling_200', 'FTL_ACROSS_TRACK_rolling_2h', 'FTL_D4PNPO', 'FTL_D4PNPO_rolling_1h', 'FTL_D4PNPO_rolling_2h', 'FTL_EARTH', 'FTL_EARTH_rolling_1h', 'FTL_EARTH_rolling_1h_gradient', 'FTL_EARTH_rolling_2h', 'FTL_INERTIAL', 'FTL_INERTIAL_rolling_1h', 'FTL_INERTIAL_rolling_2h', 'FTL_MAIN

In [4]:
for feature in sorted(shared_features):
    print feature, "diff of means", new_data[feature].mean() - old_data[feature].mean()

DMOP_event_counts_log diff of means 0.0
DMOP_event_counts_rolling_2h_gradient diff of means 0.0
DMOP_event_counts_rolling_5h diff of means 0.0
EVTF_IN_MAR_UMBRA diff of means 0.0
EVTF_IN_MAR_UMBRA_rolling_1h diff of means 0.0
EVTF_IN_MAR_UMBRA_rolling_1h_rolling_50 diff of means 0.0
EVTF_IN_MRB_/_RANGE_06000KM diff of means 0.0
EVTF_IN_MRB_/_RANGE_06000KM_gradient diff of means 0.0
EVTF_IN_MRB_/_RANGE_06000KM_rolling_1h diff of means 0.0
EVTF_IN_MRB_/_RANGE_06000KM_rolling_1h_gradient diff of means 0.0
EVTF_IN_MRB_/_RANGE_06000KM_rolling_1h_rolling_1600 diff of means 0.0
EVTF_IN_MSL_/_RANGE_06000KM diff of means 0.0
EVTF_IN_MSL_/_RANGE_06000KM_rolling_1h diff of means 0.0
EVTF_altitude diff of means 0.0
EVTF_event_counts diff of means 0.0
EVTF_event_counts_rolling_2h diff of means 0.0
EVTF_event_counts_rolling_5h diff of means 0.0
EVTF_event_counts_rolling_5h_rolling_50 diff of means 0.0
FTL_ACROSS_TRACK diff of means 0.0
FTL_ACROSS_TRACK_rolling_1h diff of means 0.0
FTL_ACROSS_TRACK_r